## This notebook performs downsampling on the OpenFace Extracted features' dataframes

1. Downsampling - to a specified ```sampling_size```
2. Preprocesses the downsampled data
    - Excludes datapoints in ```failure videos``` which are before the ```failureOccurrence_timestamp```
3. Merges all ```participants``` all ```videos``` that are downsampled to a particular ```sampling_size``` and preprocessed (as per #2)

In [1]:
import os
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

### Specify paths

- ```downsampling_directory``` - path to the directory where the downsampled ```df's``` are stored for each ```participant``` 
    - the directories are then created based on the convention of ```/{frequency}fps_downsampling/```  

```participant_features_directory``` - path where the original openFace extracted features of the participants are stored locally  
```participant_database``` - stores the list of all the participants

In [2]:
#now, open openface_features files for each participant
files_to_ignore = ['.DS_Store', '5702']
participant_features_directory = '../../studyData/openface_datasets/facial_features_openface_cut_dataset_1s/'
participant_database = [participant_folder for participant_folder in os.listdir(participant_features_directory) if participant_folder not in files_to_ignore]
len(participant_database)

29

### Downsampling


- ```required_features``` - list of all the features that are to be considered and downsampled
- ```final_features``` - list of ```metadata columns + required_features```
- ```participant_folder``` - contains the list of all the extracted features dataframe of the ```response_video```s of the ```participant```
- ```csv_file``` - openface extracted feature files of the ```response_video``` of a ```participant```
- ```target_class``` - OHE value of the class the ```response_video``` belongs to.
- ```required_df``` - a dataframe that contains all the ```final_features``` from the original ```df``` dataframe


- ```sampling_size``` - indicates how many ```datapoints in a second``` is to be considered and retained
- ```frequency = 1 / sampling_size```
- ```last_timestamp``` - last recorded time of final extracted frame of the participant's ```response_video``` in seconds
- ```valid_timestamps``` - list of all the ```sampling_size``` intervals
- ```downsampled_df``` - final ```df``` that contains downsampled data with ```final_features``` values
- ```(from_time, time]``` - time range of the datapoints based on the original_df that is to be considered for the window
- ```dataPoints_within_window``` - dataPoints within the timerange to be considered for downsampling to a single datapoint

In [7]:
import warnings

# Filter out the specific warning category
warnings.filterwarnings('ignore')

required_features = ['timestamp', 'gaze_0_x','gaze_0_y','gaze_0_z','gaze_1_x','gaze_1_y','gaze_1_z','gaze_angle_x','gaze_angle_y','pose_Tx', 'pose_Ty', 'pose_Tz','pose_Rx', 'pose_Ry', 'pose_Rz','AU01_r','AU02_r','AU04_r','AU05_r','AU06_r','AU07_r','AU09_r','AU10_r','AU12_r','AU14_r','AU15_r','AU17_r','AU20_r','AU23_r','AU25_r','AU26_r','AU45_r','AU01_c','AU02_c','AU04_c','AU05_c','AU06_c','AU07_c','AU09_c','AU10_c','AU12_c','AU14_c','AU15_c','AU17_c','AU20_c','AU23_c','AU25_c','AU26_c','AU28_c','AU45_c']
# Some initial information regarding the participant and their corresponding responseVideo information to be included
final_features = ['participant_id', 'response_video', 'class']

sampling_size = float(input('Enter the sampling size: '))
# round it to the nearest Xth second (frequency = 1/sampling_size)
frequency = int(1 / sampling_size)

video_clipping_duration = 1
downsampling_directory = f"../../studyData/openface_datasets/downsampled_feature_data/" + f"facial_features_openface_cut_dataset_{video_clipping_duration}s_{frequency}fps_downsampling/"

if not os.path.exists(downsampling_directory):
    os.makedirs(downsampling_directory)

with tqdm(total=len(participant_database)) as pbar:
    class_0 = 0
    class_1 = 0
    for participant in sorted(participant_database):
        total_participant_frames = 0
        num_data_points_below_confidence_score = 0

        participant_folder_path = f'{participant_features_directory}{participant}/'
        participant_folder = os.listdir(participant_folder_path)
        for csv_file in sorted(participant_folder):

            if csv_file in files_to_ignore or "warmup" in csv_file:
                continue

            # Identify the class to which the csv_file belongs to
            target_class = int(csv_file.split(".")[0].split("_")[-1])
            responseVideo_name = csv_file.split("_")[0] + "_" + csv_file.split("_")[1]

            # Read the features csv file
            csv_file_path = f'{participant_folder_path}{csv_file}'
            df = pd.read_csv(csv_file_path)
            
#             ## Check number of datapoints below confidence level
#             num_data_points_below_confidence_score += (df['confidence'] < 0.80).sum()
#             total_participant_frames += df.shape[0]
            
            ## Check the number of datapoint (frame) classes
            if target_class == 0:
                class_0 += 1
            else:
                class_1 += 1
            
            # Create a new dataframe - 'required_df' that retains all the feature columns that are required from the original dataframe - 'df'
            required_df = df[required_features].copy()

            # add participant column
            required_df['participant_id'] = participant
            required_df['class'] = target_class
            required_df['response_video'] = responseVideo_name

            # Reorganize the column order
            required_df = required_df[final_features + required_features]

            ## TODO: Downsampling

            # Obtain the last_timestamp for every response_video data
            last_timestamp = required_df['timestamp'].values[-1] # NOTE: this is in seconds
            last_timestamp = round(last_timestamp * frequency) / frequency
            # Now we get a list of timestamps once every 'sampling_size' seconds, until the last timestamp
            valid_timestamps = np.arange(0, last_timestamp + sampling_size, sampling_size)

            # A new dataframe to store the downsampled datapoints from the 'required_df'
            downsampled_df = pd.DataFrame()

            for time in valid_timestamps:
                if time == valid_timestamps[0]:
                    # Retrieve the rows between the range (from_time, time]
                    downsampled_df = pd.concat([downsampled_df, required_df[required_df['timestamp'] <= time]])
                    from_time = time
                else:
                    dataPoints_within_window = required_df[(required_df['timestamp'] > from_time) & (required_df['timestamp'] <= time)]

                    # Check if there exists any datapoints within the range specified: (from_time, time]
                    if len(dataPoints_within_window) > 0:

                        # Now that you have datapoints within the specified window range: (from_time, time]
                        # Calculate some statistics on the feature columns present in the dataframe
                        # i.e: mean(), max(), etc.. on the respective feature columns

                        # NOTE: here we reduce the range of dataPoints to a single dataPoint after calculating the statistics
                        for column in dataPoints_within_window.columns:

                            # for the feature columns that contain values based on classification
                            # i.e: AU##_c :- columns, find the max value amongst them (i.e: 0 or 1)
                            if '_c' in column:
                                dataPoints_within_window[column] = dataPoints_within_window[column].max()
                            # for all other columns except few, calculate the aggregate value
                            elif column not in ['participant_id', 'class', 'response_video', 'timestamp']:
                                dataPoints_within_window[column] = dataPoints_within_window[column].mean()
                            elif column == 'timestamp':
                                dataPoints_within_window[column] = time

                        # Now we add the dataPoints_within_window :- that have been reduced to a single datapoint
                        # to the final - downsampled_df
                        downsampled_df = pd.concat([downsampled_df.T, dataPoints_within_window.iloc[0, :]], axis = 1).T
                    else:
                        print(f'No datapoints within the specified range! ({from_time}, {time}]')

                    # Update the range of the time :- to shift the window
                    from_time = time
                # print(downsampled_df.shape)
            
            #make new folder in data/openFace_features_data directory, called {frequency}fps_downsampling
            downsampling_directory = downsampling_directory
            if not os.path.exists(downsampling_directory):
                os.mkdir(downsampling_directory)
            
            # If the participant directory does not exist for the downsampling, create a directory
            if not os.path.exists(f'./{downsampling_directory}/{participant}/'):
                os.mkdir(f'./{downsampling_directory}/{participant}/')

            # Save the downsampled features file as - ch1_5fps.csv in the directory: participant
            downsampled_df.to_csv(f'./{downsampling_directory}/{participant}/{responseVideo_name}_{frequency}fps.csv', index = False)
        
#         num_datapoints_below_confidence_percentage = num_data_points_below_confidence_score / total_participant_frames * 100
#         print(f"Participant {participant}: Datapoints below 80% confidence level = {num_datapoints_below_confidence_percentage:.03f}% ({num_data_points_below_confidence_score} / {total_participant_frames})")
        pbar.update(1)
        pbar.set_description(f'Participant ID = {participant}')
    print("Number of Frames: ")
    print(f"Class 0: {class_0}\nClass 1: {class_1}")

# After you're done with the code, you can reset the warning filters
warnings.resetwarnings()

Enter the sampling size: 0.2


Participant ID = 9941: 100%|████████████████████| 29/29 [00:24<00:00,  1.17it/s]

Number of Frames: 
Class 0: 463
Class 1: 402


#### Merge

- Here we merge - all ```participants``` all ```responseVideos``` into a single dataframe called as ```allParticipants_frequencyfps_downsampled_preprocessed``` 

In [8]:
downsampled_participant_database = os.listdir(downsampling_directory)
downsampled_participant_database = [participant_folder for participant_folder in downsampled_participant_database if participant_folder not in files_to_ignore]

In [10]:
allParticipants_df = pd.DataFrame()

with tqdm(total=len(downsampled_participant_database)) as pbar:
    for participant in sorted(downsampled_participant_database):
        participant_folder_path = f'{downsampling_directory}{participant}/'
        participant_folder = os.listdir(participant_folder_path)

        for csv_file in sorted(participant_folder):
            if csv_file in files_to_ignore:
                continue
            current_df = pd.read_csv(f'{participant_folder_path}{csv_file}')
            allParticipants_df = pd.concat([allParticipants_df, current_df])
        pbar.update(1)
        pbar.set_description(f'Participant ID = {participant}')
allParticipants_df.to_csv(f'{downsampling_directory}allParticipants_{frequency}fps_downsampled_preprocessed.csv', index = False)

Participant ID = 9941: 100%|████████████████████| 29/29 [00:01<00:00, 27.08it/s]


#### Here, we normalize the features

In [11]:
allParticipants_df_norm = copy.deepcopy(allParticipants_df)

columns_to_normalize = allParticipants_df_norm.columns[4:]
scaler = StandardScaler()
allParticipants_df_norm[columns_to_normalize] = scaler.fit_transform(allParticipants_df_norm[columns_to_normalize])

allParticipants_df_norm.to_csv(f'{downsampling_directory}/allParticipants_{frequency}fps_downsampled_preprocessed_norm.csv', index = False)